In [1]:
example_text_list = [
    "Bilbo Baggins celebrates his birthday and leaves the Ring to Frodo, his heir. Gandalf (a wizard) suspects it is a Ring of Power; seventeen years later, he confirms it was lost by the Dark Lord Sauron and counsels Frodo to take it away from the Shire. Gandalf leaves, promising to return, but fails to do so. Frodo sets out on foot with his cousin Pippin Took and gardener Sam Gamgee. They are pursued by Black Riders, but meet some Elves, whose singing to Elbereth wards off the Riders. The Hobbits take an evasive shortcut to Bucklebury Ferry, where they meet their friend Merry Brandybuck. Merry and Pippin reveal they know about the Ring and insist on joining Frodo on his journey. They try to shake off the Black Riders by cutting through the Old Forest. Merry and Pippin are trapped by the malign Old Man Willow, but are rescued by Tom Bombadil. Leaving Tom's house, they are caught by a barrow-wight. Frodo, awakening from the barrow-wight's spell, calls Tom Bombadil, who frees them and gives them ancient swords from the wight's hoard. The Hobbits reach the village of Bree, where they meet Strider, a Ranger. The innkeeper gives Frodo an old letter from Gandalf, which identifies Strider as a friend. Knowing the Black Riders will attempt to seize the Ring, Strider guides the group toward the Elvish sanctuary of Rivendell. At Weathertop, they are attacked by five Black Riders. Their leader wounds Frodo with a cursed blade. Strider fights them off and treats Frodo with the herb athelas. They are joined by the Elf Glorfindel, who rides with Frodo, now deathly ill, towards Rivendell. The Black Riders pursue Frodo into the Ford of Bruinen, where they are swept away by flood waters summoned by Elrond.",
    "Frodo recovers in Rivendell under Elrond's care. Gandalf informs Frodo that the Black Riders are the Nazgûl, Men enslaved by Rings of Power to serve Sauron. The Council of Elrond discusses what to do with the Ring. Strider is revealed to be Aragorn, the heir of Isildur who had cut the Ring from Sauron's hand in the Second Age, but claimed it for himself. The Ring was lost when Isildur was killed; it passed to Gollum and then to Bilbo. Gandalf reports that the chief wizard, Saruman, is a traitor. The Council decides that the Ring must be destroyed in the fire of Mount Doom in Mordor, where it was forged. Frodo takes this task upon himself. Elrond chooses companions for him: Sam, Merry, and Pippin; Gandalf; the Men Aragorn and Boromir, son of the Steward of Gondor; the Elf Legolas; and the Dwarf Gimli, representing the Free Peoples of the West. After a failed attempt to cross the Misty Mountains, the Fellowship risk the path through the Mines of Moria. They learn that Balin and his Dwarves were killed by Orcs. They are attacked by Orcs and a Balrog, a fire demon. Gandalf confronts the Balrog: both fall into an abyss. The others escape to the Elvish forest of Lothlórien, where the Lady Galadriel tests their loyalty, and gives them magical gifts. She allows Frodo and Sam to look into her vision-giving fountain, the Mirror of Galadriel. Frodo offers her the Ring: she refuses, knowing that it would master her. Galadriel's husband Celeborn gives the Fellowship boats, cloaks, and waybread. They travel down the River Anduin. At Amon Hen, Boromir tries to take the Ring, but Frodo puts on the Ring and disappears. Frodo chooses to cross the river and go alone to Mordor, but Sam, guessing what he intends, intercepts him.",
    # "A party of Orcs sent by Saruman and Sauron attack the Fellowship. Boromir tries to protect Merry and Pippin from the Orcs, but they kill him and capture the two Hobbits. Aragorn, Gimli and Legolas decide to pursue the Orcs. The Orcs are killed by Riders of Rohan, led by Éomer. The Hobbits escape into Fangorn Forest, where they are befriended by the Ent Treebeard. Aragorn, Gimli and Legolas track the hobbits to Fangorn, where they meet Gandalf: he explains that he killed the Balrog; he too was killed in the fight, but was sent back to Middle-earth. He is now Gandalf the White, replacing Saruman as chief of the wizards. They ride to Edoras, capital of Rohan. Gandalf frees King Théoden from the influence of Saruman's spy Gríma Wormtongue. Théoden musters his army and rides to the fortress of Helm's Deep; Gandalf departs to seek help from Treebeard. The Ents destroy Isengard, Saruman's stronghold, and flood it, trapping the wizard in the tower of Orthanc. Gandalf convinces Treebeard to send an army of Huorns to Théoden's aid. He brings an army of Riders of Rohan to Helm's Deep. They defeat the Orcs, who flee into the forest of Huorns and are destroyed. Gandalf, Théoden, Aragorn, Legolas, and Gimli ride to Isengard, where they find Merry and Pippin relaxing amidst the ruins. Gandalf offers Saruman a chance to turn away from evil. When Saruman refuses, Gandalf strips him of his rank and most of his powers. Wormtongue throws down a hard round object to try to kill Gandalf. Pippin picks it up; Gandalf swiftly takes it, but Pippin steals it in the night. It is a palantír, a seeing-stone that Saruman used to speak with Sauron, becoming ensnared. Sauron sees Pippin, but misunderstands the circumstances. Gandalf rides for Minas Tirith, chief city of Gondor, taking Pippin with him.",
    # "Frodo and Sam struggle through the barren hills of the Emyn Muil. They realize they are being tracked; on a moonlit night they capture Gollum, who has followed them from Moria. Frodo makes Gollum swear to serve him, as Ringbearer, and asks him to guide them to Mordor. Gollum leads them across the Dead Marshes. Sam overhears Gollum debating with his alter ego, Sméagol, whether to steal the Ring. The Black Gate of Mordor is too well guarded, so they travel south through Ithilien to a secret pass that Gollum knows. They are captured by rangers led by Faramir, Boromir's brother, and brought to the secret fastness of Henneth Annûn. Faramir resists the temptation to seize the Ring and, disobeying orders to arrest strangers, releases them. Gollum guides the hobbits to the pass, but leads them into the lair of the great spider Shelob in the tunnels of Cirith Ungol. Frodo holds up his gift, the Phial of Galadriel, which holds the light of Eärendil's star: it drives Shelob back. Frodo cuts through a giant web using his sword Sting. Shelob attacks again, and Frodo falls to her venom. Sam picks up Sting and the Phial. He wounds the monster. Believing Frodo to be dead, Sam takes the Ring to continue the quest alone. Orcs take Frodo; Sam overhears them saying that Frodo is still alive.",
    # "Sauron sends a great army against Gondor. Gandalf arrives at Minas Tirith to warn Denethor of the attack, while Théoden musters the Riders of Rohan to go to Gondor's aid. Minas Tirith is besieged; the Lord of the Nazgûl uses a spell-wound battering ram to destroy the city's gates. Denethor, deceived by Sauron, falls into despair. He burns himself alive on a pyre; Pippin and Gandalf rescue his son Faramir from the same fate. Aragorn, accompanied by Legolas, Gimli, and the Rangers of the North, takes the Paths of the Dead to recruit the Dead Men of Dunharrow, oathbreakers who will have no rest until they fight for the King of Gondor. Aragorn unleashes the Army of the Dead on the Corsairs of Umbar invading southern Gondor. He and men of southern Gondor sail in the Corsairs' ships up the Anduin, reaching Minas Tirith just in time to turn the tide of battle. Théoden's niece Éowyn, who joined the army in disguise, kills the Lord of the Nazgûl with help from Merry; both are wounded. Gondor and Rohan defeat Sauron's army in the Battle of the Pelennor Fields, but Théoden is killed. Aragorn enters Minas Tirith and heals the sick. He leads an army through Ithilien to the Black Gate to distract Sauron from his true danger. At the Battle of the Morannon, his army is vastly outnumbered.",
    # "Sam rescues Frodo from the tower of Cirith Ungol. They set out across Mordor. When they reach the edge of the Cracks of Doom, Frodo cannot resist the Ring any longer: he claims it for himself and puts it on. Gollum reappears. He bites off Frodo's Ring finger. Celebrating wildly, Gollum loses his footing and falls into the Fire, taking the Ring with him. When the Ring is destroyed, Sauron loses his power. All he created collapses, the Nazgûl perish, and Aragorn wins the battle of the Morannon. Aragorn is crowned King, and weds Arwen, Elrond's daughter. Théoden is buried; Éomer is crowned King of Rohan. His sister Éowyn is engaged to Faramir, now Steward of Gondor and Prince of Ithilien. Galadriel, Celeborn, and Gandalf say farewell to Treebeard, and to Aragorn. The four hobbits travel home, only to find it has been taken over by Saruman's men. Merry raises a rebellion and scours the Shire. Wormtongue turns on Saruman and kills him in front of Bag End, Frodo's home; he is killed by hobbit archers. Merry and Pippin are celebrated as heroes. Sam marries Rosie Cotton and uses his gifts to heal the Shire. Frodo, broken by the quest, leaves a few years later, sailing from the Grey Havens over the Sea to find peace.",
    # "Extensive appendices outline more details of the history, cultures, genealogies, and languages that Tolkien imagined for the peoples of Middle-earth. In antiquarian style,[6] they provide background details for the narrative, with much detail for Tolkien fans who want to know more about the stories.",
]

# example_text_list = [
# """Vyasa had composed a poem and needed a scribe to write it down.
# "Will you be my scribe?" he asked Ganesha, the elephant-headed god.
# "I will," said Ganesha, "provided you do not pause in your recitation."
# "I agree," Vyasa replied, "provided you understand each word's meaning before you write it down."
# Vyasa recited, and Ganesha wrote.
# Sometimes Vyasa said things that were confusing, and Ganesha would pause and think.
# Once, when Ganesha's pen splintered, he broke off one of his own tusks to keep writing. 
# That was the first version of the Mahabharata.
# This Mahabharata will begin with King Shantanu."""
# ,
# """
# King Shantanu lived in a great palace in Hastinapura, but he had no queen.
# One day he went hunting, and by the river he saw a beautiful woman. He loved the woman at first sight. "Marry me!" he said.
# "I agree," the woman replied, "under one condition: you must never question my actions."
# Shantanu agreed, and they were soon married.
# A year later, their first child was born.
# On that very day, the queen took the baby to the river and drowned him there.
# King Shantanu was stunned, but said nothing.
# He kept his promise to never question her actions.
# """
# ,
# """
# To understand Shantanu's story, listen to the story of Mahabhisha:
# King Mahabhisha had earned so much merit that he ascended to Indra's heaven. There he danced with the apsaras to the music of the gandharvas, drinking sura, and the wish-granting tree Kalpataru gave him everything he desired.
# One day a breeze blew the goddess Ganga's garment aside. The devas looked away, but Mahabhisha could not resist: he stared directly at her naked breasts. 
# Indra cursed Mahabhisha to return to earth, and he told Ganga to take human birth and break his heart.
# Mahabhisha was reborn as Shantanu, king of Hastinapura.
# """
# ,
# """
# King Shantanu's queen drowned their first child in the river, and their second. One by one, she drowned seven children.
# Shantanu said nothing.
# When she gave birth to their eighth child, Shantanu shouted, "Stop! I forbid you to kill this child."
# "The child will live, but I must leave you now," the queen replied. "I am Ganga, goddess of this river, and I came to earth in order to marry you, bear your children, and drown them as soon as they were born. Our children were the eight Vasus, gods of the eight elements, cursed to be born as humans."
# """
# ,
# """
# Prabhasa, chief of the Vasus, had coveted the wish-granting cow of the rishi Vashishtha. Together with the other Vasus, Prabhasa stole the cow, but Vashishtha caught them and cursed them. "You will be born on earth as humans."
# The Vasus begged Ganga to be their mother on earth and drown them, making their lives as short as possible.
# Ganga agreed. She became the wife of King Shantanu, and she drowned their children as soon as they were born. But when the eighth child was born, Shantanu stopped her. 
# That eighth child was the incarnation of Prabhasa.
# Shantanu named him Devavrata.
# """
# ,
# """
# "I will take Devavrata with me now," Ganga told Shantanu, cradling the infant in her arms, "and return him to you later." She then disappeared into the river.
# King Shantanu returned every day to the river, forever hoping that his wife and son would appear. 
# Then, one day, it happened: Ganga emerged from the river, together with a handsome young man. "Your son has learned the Vedas from the rishi Vashishtha," she said, "and Parashurama has taught him the arts of war." 
# Shantanu embraced his son and proclaimed him to be the crown-prince, his heir.
# Ganga then returned to heaven.
# """
# ,
# """
# King Shantanu went hunting one day.
# Again, he saw a beautiful woman by the river.
# Again, he fell in love at first sight.
# "Marry me!" he said.
# "You must ask my father's permission," she replied. 
# "Who are you?" Shantanu asked. "And who is your father?"
# "I am Satyavati," she said. "My father is a fisherman."
# "Let me marry your daughter!" Shantanu said to the fisherman.
# "I agree," he said, "under one condition: when Satyavati bears you a son, that son must inherit the kingdom."
# But Shantanu could not agree to this condition because he had named Devavrata as his heir.
# """
# ]

In [2]:
from graph_maker import GraphMaker, Ontology, GroqClient
from graph_maker import Document

In [3]:
# model = "mixtral-8x7b-32768"
model ="llama3-8b-8192"
# model = "llama3-70b-8192"
# model="gemma-7b-it"

ontology = Ontology(
    label=[
        {"Person": "Person name without any adjectives, Remember a person may be references by their name or using a pronoun"},
        {"Object": "Do not add the definite article 'the' in the object name"},
        {"Event": "Event event involving multiple people. Do not include qualifiers or verbs like gives, leaves, works etc."},
        "Place",
        "Document",
        "Organisation",
        "Action",
        {"Miscellanous": "Any important concept can not be categorised with any other given label"},
    ],
    relationships=[
        "Relation between any pair of Entities"
        ],
)


In [4]:
import datetime
current_time = str(datetime.datetime.now())
llm = GroqClient(model=model, temperature=0.1, top_p=0.5)
graph_maker = GraphMaker(ontology=ontology, llm_client=llm, verbose=False)

def generate_summary(text):
    SYS_PROMPT = (
        "Succintly summarise the text provided by the user. "
        "Respond only with the summary and no other comments"
    )
    try:
        summary = llm.generate(user_message=text, system_message=SYS_PROMPT)
    except:
        summary = ""
    finally:
        return summary


docs = map(
    lambda t: Document(text=t, metadata={"summary": generate_summary(t), 'generated_at': current_time}),
    example_text_list
)


In [5]:

graph = graph_maker.from_documents(list(docs))
print("Total number of Edges", len(graph))

Using Model:  llama3-8b-8192


INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


Using Model:  llama3-8b-8192


INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"

▶︎ GRAPH MAKER LOG - 2024-05-01 13:28:35 - INFO 
Document: 1


▶︎ GRAPH MAKER VERBOSE - 2024-05-01 13:28:35 - INFO 
Using Ontology:
label=[{'Person': 'Person name without any adjectives, Remember a person may be references by their name or using a pronoun'}, {'Object': "Do not add the definite article 'the' in the object name"}, {'Event': 'Event event involving multiple people. Do not include qualifiers or verbs like gives, leaves, works etc.'}, 'Place', 'Document', 'Organisation', 'Action', {'Miscellanous': 'Any important concept can not be categorised with any other given label'}] relationships=['Relation between any pair of Entities']



Using Model:  llama3-8b-8192


INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"

▶︎ GRAPH MAKER VERBOSE - 2024-05-01 13:28:36 - INFO 
LLM Response:
[
    {
        "node_1": {
            "label": "Person",
            "name": "Bilbo Baggins"
        },
        "node_2": {
            "label": "Person",
            "name": "Frodo"
        },
        "relationship": "Bilbo leaves the Ring to Frodo, his heir."
    },
    {
        "node_1": {
            "label": "Person",
            "name": "Gandalf"
        },
        "node_2": {
            "label": "Person",
            "name": "Frodo"
        },
        "relationship": "Gandalf confirms it was lost by the Dark Lord Sauron and counsels Frodo to take it away from the Shire."
    },
    {
        "node_1": {
            "label": "Person",
            "name": "Gandalf"
        },
        "node_2": {
            "label": "Person",
            "name": "Frodo"
        },
        "relationship": "Gandalf fails to return as 

Using Model:  llama3-8b-8192


INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"

▶︎ GRAPH MAKER VERBOSE - 2024-05-01 13:28:38 - INFO 
LLM Response:
[
    {
        "node_1": {
            "label": "Person",
            "name": "Frodo"
        },
        "node_2": {
            "label": "Person",
            "name": "Elrond"
        },
        "relationship": "Frodo recovers in Rivendell under Elrond's care."
    },
    {
        "node_1": {
            "label": "Person",
            "name": "Gandalf"
        },
        "node_2": {
            "label": "Person",
            "name": "Frodo"
        },
        "relationship": "Gandalf informs Frodo that the Black Riders are the Nazgûl."
    },
    {
        "node_1": {
            "label": "Person",
            "name": "Gandalf"
        },
        "node_2": {
            "label": "Person",
            "name": "Saruman"
        },
        "relationship": "Gandalf reports that Saruman is a traitor."
    },
    {
        "nod

Total number of Edges 22


# Saving to Neo4j 

In [6]:
from graph_maker import Neo4jGraph

create_indices = False
neo4j_graph = Neo4jGraph(edges=graph, create_indices=create_indices)

neo4j_graph.save()


aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called


22